<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap plotly xgboost --quiet

In [2]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00


In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [6]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MJxCIqPSTQoAnA_Zt9u3FjjvXlWa74pzpC78XDuhQ9o/edit?usp=sharing')
worksheet = gsheets.worksheet("V6_expansion")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1200,NaN,NaN,NaN,0,180,1,80,0,45,20,45,25,17.677670,1,0.0,0.0,0.0,1.0,1
1,1252,NaN,NaN,NaN,0,180,1,80,0,55,20,55,35,12.738631,1,0.0,0.0,0.0,1.0,1
2,1178,NaN,NaN,NaN,0,180,0,81,0,10,10,10,0,NaN,0,0.0,0.0,0.0,1.0,1
3,1286,NaN,NaN,NaN,0,180,1,79,0,40,15,40,25,12.041595,1,0.0,0.0,0.0,1.0,1
4,1298,NaN,NaN,NaN,0,180,1,77,0,65,50,65,15,6.634888,1,0.0,1.0,0.0,1.0,1


In [7]:
external['呼吸_ADL低'] = ((external['使用呼吸輔具'] == 1) & (external['ADL_總分_max'] < 30)).astype(int)
external['高齡_住院多'] = ((external['預估年齡'] > 85) & (external['六個月內住院次數'] > 2)).astype(int)
external['ADL低_有管路'] = ((external['ADL_總分_max'] < 30) & (external['使用管路'] == 1)).astype(int)
external['DNR_呼吸'] = ((external['DNR_flag'] == 1) & (external['使用呼吸輔具'] == 1)).astype(int)
external['超級重症'] = (
    (external['DNR_flag'] == 1) &
    (external['使用呼吸輔具'] == 1) &
    (external['ADL_總分_max'] < 30) &
    (external['六個月內住院次數'] > 1)
).astype(int)
external["low_adl_no_var"] = ((external["ADL_std"].isnull()) & (external["ADL_總分_max"] < 5)).astype(int)

In [8]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,9827.0,1508.933652,3979.829302,1.0,1174.0,1234.0,1330.000000,100462.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,9827.0,0.412740,0.492352,0.0,0.0,0.0,1.000000,1.000000
觀察天數,9827.0,134.395543,63.203515,0.0,78.0,180.0,180.000000,180.000000
性別_is_male,9827.0,0.522336,0.499526,0.0,0.0,1.0,1.000000,1.000000
預估年齡,9827.0,79.366236,11.612847,0.0,73.0,81.0,88.000000,125.000000
DNR_flag,9827.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,9827.0,20.425359,28.851272,0.0,0.0,5.0,30.000000,100.000000


In [9]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


訓練資料讀入

In [11]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UFokE2ATcNkejRy4PSCkcbCeYzrxOslhelzpd2OaPlc/edit?usp=sharing')
worksheet = gsheets.worksheet("V6")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1175,NaN,NaN,NaN,0,180,1,82,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,5
1,800,NaN,NaN,NaN,0,180,1,79,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,2
2,1262,NaN,NaN,NaN,0,180,0,77,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,3
3,1297,NaN,NaN,NaN,0,180,0,80,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,3
4,1240,NaN,NaN,NaN,0,180,1,82,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,4


In [12]:
df['呼吸_ADL低'] = ((df['使用呼吸輔具'] == 1) & (df['ADL_總分_max'] < 30)).astype(int)
df['高齡_住院多'] = ((df['預估年齡'] > 85) & (df['六個月內住院次數'] > 2)).astype(int)
df['ADL低_有管路'] = ((df['ADL_總分_max'] < 30) & (df['使用管路'] == 1)).astype(int)
df['DNR_呼吸'] = ((df['DNR_flag'] == 1) & (df['使用呼吸輔具'] == 1)).astype(int)
df['超級重症'] = (
    (df['DNR_flag'] == 1) &
    (df['使用呼吸輔具'] == 1) &
    (df['ADL_總分_max'] < 30) &
    (df['六個月內住院次數'] > 1)
).astype(int)
df["low_adl_no_var"] = ((df["ADL_std"].isnull()) & (df["ADL_總分_max"] < 5)).astype(int)

In [13]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,8782.0,1579.687657,4465.434171,6.0,1178.00,1243.0,1345.000000,100462.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,8782.0,0.477568,0.499525,0.0,0.00,0.0,1.000000,1.000000
觀察天數,8782.0,128.299362,64.349164,1.0,65.25,180.0,180.000000,180.000000
性別_is_male,8782.0,0.516283,0.499763,0.0,0.00,1.0,1.000000,1.000000
預估年齡,8782.0,78.948531,11.752762,0.0,72.00,81.0,87.000000,123.000000
DNR_flag,8782.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000
ADL_總分_max,8782.0,24.066272,30.736311,0.0,0.00,10.0,40.000000,100.000000


In [14]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


In [15]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.5].index.tolist()

In [16]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'ADL_first_score',
 'ADL_總分_max',
 'DNR_flag',
 'ADL_diff',
 'ADL_明顯惡化',
 'ADL_last_score',
 'DNR_呼吸',
 '呼吸_ADL低',
 '高齡_住院多',
 'ADL低_有管路',
 '六個月內住院次數',
 'low_adl_no_var',
 '超級重症',
 '活動假牙',
 '使用管路',
 '意識清醒_max',
 '使用呼吸輔具',
 'ADL_std']

In [17]:
dfNew = df[features]

In [18]:
dfNew = dfNew.fillna(0)

In [19]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.6,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.02,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=3,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=500,         # ✅ 總樹數可略減以免累積錯誤
            subsample=0.8,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=300,
            max_depth=5,             # ✅ 控制單棵樹深度，避免全擬合
            max_features='sqrt',     # ✅ 每次切分隨機抽部分特徵
            min_samples_leaf=10,     # ✅ 每個葉節點最少樣本，避免太小造成過擬合
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [20]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [21]:
dfNew

,性別_is_male,死亡標記,預估年齡,ADL_first_score,ADL_總分_max,DNR_flag,ADL_diff,ADL_明顯惡化,ADL_last_score,DNR_呼吸,...,高齡_住院多,ADL低_有管路,六個月內住院次數,low_adl_no_var,超級重症,活動假牙,使用管路,意識清醒_max,使用呼吸輔具,ADL_std
0,1,0,82,0,0,0,0,0,0,0,...,0,1,5,0,0,0.0,1.0,0.0,0.0,0.0
1,1,0,79,0,0,0,0,0,0,0,...,0,1,2,0,0,0.0,1.0,0.0,0.0,0.0
2,0,0,77,0,0,0,0,0,0,0,...,0,1,3,0,0,0.0,1.0,0.0,0.0,0.0
3,0,0,80,0,0,0,0,0,0,0,...,0,1,3,0,0,0.0,1.0,0.0,0.0,0.0
4,1,0,82,0,0,0,0,0,0,0,...,0,1,4,0,0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8777,0,1,69,40,40,0,0,0,40,0,...,0,0,2,0,0,0.0,1.0,0.0,1.0,0.0
8778,1,1,88,0,0,0,0,0,0,0,...,0,1,2,0,0,0.0,1.0,0.0,0.0,0.0
8779,1,1,72,0,0,0,0,0,0,0,...,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0
8780,1,1,85,25,25,0,0,0,25,0,...,0,1,1,0,0,0.0,1.0,0.0,0.0,0.0


In [22]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [23]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [24]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_first_score,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_diff,0,0.0
ADL_明顯惡化,0,0.0
ADL_last_score,0,0.0
DNR_呼吸,0,0.0
呼吸_ADL低,0,0.0


# 開始進行訓練

In [25]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,8782.0,0.516283,0.499763,0.0,0.0,1.0,1.000000,1.000000
預估年齡,8782.0,78.948531,11.752762,0.0,72.0,81.0,87.000000,123.000000
ADL_first_score,8782.0,15.959349,25.705937,0.0,0.0,0.0,20.000000,100.000000
ADL_總分_max,8782.0,24.066272,30.736311,0.0,0.0,10.0,40.000000,100.000000
DNR_flag,8782.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_diff,8782.0,8.106923,17.196131,0.0,0.0,0.0,5.000000,100.000000
ADL_明顯惡化,8782.0,0.245730,0.430544,0.0,0.0,0.0,0.000000,1.000000
ADL_last_score,8782.0,24.066272,30.736311,0.0,0.0,10.0,40.000000,100.000000
DNR_呼吸,8782.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
呼吸_ADL低,8782.0,0.241972,0.428302,0.0,0.0,0.0,0.000000,1.000000


In [26]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 14.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=519032f63f0c8936277a982fb66dd6294e13b400df2397da57071ad4887c405d
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lifelines import CoxPHFitter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# 假設 HybridXGBRF 已定義
all_models = {
    "HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "Alrawi (2013)": LogisticRegression(max_iter=200),
    "Chandra (2022)": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss'),
    "García-Gollarte (2020)": RandomForestClassifier(n_estimators=100, random_state=42),
    "Levy (2015)": LogisticRegression(max_iter=1000),

    # 🔽 新增未測試模型
    "Ridge": make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42)),
    "Lasso": make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)),
    "Elastic": make_pipeline(StandardScaler(), LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)),
}

In [28]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)
▶ Running CV for: Alrawi (2013)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.11/dist-packages/sklearn/

▶ Running CV for: Chandra (2022)
▶ Running CV for: García-Gollarte (2020)
▶ Running CV for: Levy (2015)
▶ Running CV for: Ridge
▶ Running CV for: Lasso


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Elastic


In [29]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [30]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,HybridXGBRF (Our Approach),0.761673,0.005930,0.760636,0.731282,0.745512,0.849250,0.007254,"[[3622, 966], [1127, 3067]]"
1,Chandra (2022),0.754385,0.007704,0.761957,0.706484,0.733095,0.836191,0.008112,"[[3662, 926], [1231, 2963]]"
2,Levy (2015),0.709860,0.011636,0.703738,0.678587,0.690837,0.799201,0.012934,"[[3388, 1200], [1348, 2846]]"
3,Lasso,0.709063,0.011978,0.703345,0.676680,0.689652,0.798711,0.012974,"[[3389, 1199], [1356, 2838]]"
4,Elastic,0.708721,0.011797,0.702903,0.676441,0.689320,0.798473,0.012973,"[[3387, 1201], [1357, 2837]]"
5,Ridge,0.708493,0.011979,0.702752,0.675964,0.689003,0.798194,0.012984,"[[3387, 1201], [1359, 2835]]"
6,García-Gollarte (2020),0.724323,0.005718,0.723843,0.683835,0.703223,0.797886,0.008259,"[[3493, 1095], [1326, 2868]]"
7,Alrawi (2013),0.708153,0.008797,0.697408,0.687647,0.692385,0.797668,0.013271,"[[3335, 1253], [1310, 2884]]"


# 測試外部資料在 XGBoost 模型下的結果

In [31]:
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數,呼吸_ADL低,高齡_住院多,ADL低_有管路,DNR_呼吸,超級重症,low_adl_no_var
0,1200,NaN,NaN,NaN,0,180,1,80,0,45,...,0.0,0.0,1.0,1,0,0,0,0,0,0
1,1252,NaN,NaN,NaN,0,180,1,80,0,55,...,0.0,0.0,1.0,1,0,0,0,0,0,0
2,1178,NaN,NaN,NaN,0,180,0,81,0,10,...,0.0,0.0,1.0,1,0,0,1,0,0,0
3,1286,NaN,NaN,NaN,0,180,1,79,0,40,...,0.0,0.0,1.0,1,0,0,0,0,0,0
4,1298,NaN,NaN,NaN,0,180,1,77,0,65,...,1.0,0.0,1.0,1,0,0,0,0,0,0


In [32]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [33]:
ex_X = ex_X.fillna(0)

In [34]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [35]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,9827.0,0.522336,0.499526,0.0,0.0,1.0,1.000000,1.000000
預估年齡,9827.0,79.366236,11.612847,0.0,73.0,81.0,88.000000,125.000000
ADL_first_score,9827.0,13.082324,23.458015,0.0,0.0,0.0,15.000000,100.000000
ADL_總分_max,9827.0,20.425359,28.851272,0.0,0.0,5.0,30.000000,100.000000
DNR_flag,9827.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_diff,9827.0,7.343034,16.565552,0.0,0.0,0.0,5.000000,100.000000
ADL_明顯惡化,9827.0,0.216241,0.411701,0.0,0.0,0.0,0.000000,1.000000
ADL_last_score,9827.0,20.425359,28.851272,0.0,0.0,5.0,30.000000,100.000000
DNR_呼吸,9827.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
呼吸_ADL低,9827.0,0.248804,0.432342,0.0,0.0,0.0,0.000000,1.000000


In [36]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_first_score,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_diff,0,0.0
ADL_明顯惡化,0,0.0
ADL_last_score,0,0.0
DNR_呼吸,0,0.0
呼吸_ADL低,0,0.0


In [37]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [38]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating Alrawi (2013)...
🔍 Evaluating Chandra (2022)...
🔍 Evaluating García-Gollarte (2020)...
🔍 Evaluating Levy (2015)...
🔍 Evaluating Ridge...
🔍 Evaluating Lasso...
🔍 Evaluating Elastic...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.750789,0.677805,0.755178,0.714402,0.836077
1,Alrawi (2013),0.672230,0.587034,0.694280,0.636169,0.759495
2,Chandra (2022),0.746515,0.684074,0.716963,0.700132,0.822206
3,García-Gollarte (2020),0.816119,0.731426,0.876233,0.797308,0.893374
4,Levy (2015),0.677012,0.594191,0.685897,0.636759,0.760930
5,Ridge,0.675893,0.593235,0.683185,0.635041,0.759413
6,Lasso,0.676707,0.594011,0.684665,0.636124,0.760103
7,Elastic,0.676707,0.594051,0.684418,0.636041,0.759753
